# README: Data Augmentation pour DialogueGCN

## Objectif
Améliorer les performances du modèle DialogueGCN en enrichissant le jeu de données existant grâce à des techniques de data augmentation, sans nécessiter de nouvelles annotations.

---

## Techniques d'Augmentation Implémentées

### 1. Génération de Paraphrases avec GPT

**Fichier:** `data_augmentation/paraphrase_generator.py`

```python
from transformers import pipeline

class ParaphraseGenerator:
    def __init__(self, model_name="gpt2"):
        self.generator = pipeline('text-generation', model=model_name)
    
    def generate_paraphrase(self, text, num_return_sequences=3):
        prompt = f"Paraphrase the following text: '{text}'\nParaphrase:"
        outputs = self.generator(
            prompt,
            num_return_sequences=num_return_sequences,
            max_length=len(text) + 20,
            temperature=0.7
        )
        return [output['generated_text'].replace(prompt, '').strip() for output in outputs]
```

**Utilisation:**

```python
generator = ParaphraseGenerator()
original_text = "Okay, I understand"
paraphrases = generator.generate_paraphrase(original_text)
```

---

### 2. Back-Translation (Traduction Double)

**Fichier:** `data_augmentation/back_translator.py`

```python
from transformers import MarianMTModel, MarianTokenizer

class BackTranslator:
    def __init__(self, intermediate_lang="fr"):
        self.intermediate_lang = intermediate_lang
        self.to_foreign = f"Helsinki-NLP/opus-mt-en-{intermediate_lang}"
        self.to_english = f"Helsinki-NLP/opus-mt-{intermediate_lang}-en"
        
        self.tokenizer_to = MarianTokenizer.from_pretrained(self.to_foreign)
        self.model_to = MarianMTModel.from_pretrained(self.to_foreign)
        self.tokenizer_back = MarianTokenizer.from_pretrained(self.to_english)
        self.model_back = MarianMTModel.from_pretrained(self.to_english)
    
    def translate(self, text, tokenizer, model):
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        outputs = model.generate(**inputs)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def back_translate(self, text):
        foreign = self.translate(text, self.tokenizer_to, self.model_to)
        return self.translate(foreign, self.tokenizer_back, self.model_back)
```

**Utilisation:**

```python
translator = BackTranslator(intermediate_lang="es")  # Espagnol comme langue intermédiaire
augmented_text = translator.back_translate("Okay, I understand")
```

---

### 3. Ajout de Bruit Audio (pour données vocales)

**Fichier:** `data_augmentation/audio_augmenter.py`

```python
import numpy as np
import librosa

class AudioAugmenter:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
    
    def add_noise(self, audio, noise_level=0.005):
        noise = np.random.randn(len(audio)) * noise_level
        return audio + noise
    
    def time_stretch(self, audio, rate=1.1):
        return librosa.effects.time_stretch(audio, rate=rate)
    
    def pitch_shift(self, audio, n_steps=2):
        return librosa.effects.pitch_shift(audio, sr=self.sample_rate, n_steps=n_steps)
    
    def augment_audio(self, audio_path, output_path):
        audio, sr = librosa.load(audio_path, sr=self.sample_rate)
        
        augmented = self.add_noise(audio)
        augmented = self.time_stretch(augmented)
        augmented = self.pitch_shift(augmented)
        
        librosa.output.write_wav(output_path, augmented, sr)
```

---

## Intégration avec DialogueGCN

### Étape 1: Préparation des Données
Placer vos données originales dans `data/original/`

Structure attendue:

```
data/original/
├── train/
├── dev/
└── test/
```

### Étape 2: Exécution de l'Augmentation

**Script:** `augment_data.py`

```python
from data_augmentation.paraphrase_generator import ParaphraseGenerator
from data_augmentation.back_translator import BackTranslator
import os
import json

def augment_dataset(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    generator = ParaphraseGenerator()
    translator = BackTranslator()
    
    for split in ['train', 'dev']:  # Ne pas augmenter le test set
        input_path = os.path.join(input_dir, split)
        output_path = os.path.join(output_dir, split)
        
        for filename in os.listdir(input_path):
            with open(os.path.join(input_path, filename)) as f:
                data = json.load(f)
                
            for item in data:
                original_text = item['text']
                
                paraphrases = generator.generate_paraphrase(original_text, 2)
                back_translated = translator.back_translate(original_text)
                
                item['augmented'] = paraphrases + [back_translated]
            
            with open(os.path.join(output_path, filename), 'w') as f:
                json.dump(data, f)

if __name__ == "__main__":
    augment_dataset("data/original", "data/augmented")
```

---

### Étape 3: Entraînement avec Données Augmentées

**Modifier le DataLoader pour utiliser les données augmentées:**

```python
class AugmentedDataset(Dataset):
    def __init__(self, data_path):
        self.data = []
        for filename in os.listdir(data_path):
            with open(os.path.join(data_path, filename)) as f:
                self.data.extend(json.load(f))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        if 'augmented' in item and random.random() > 0.5:
            text = random.choice(item['augmented'])
        else:
            text = item['text']
        
        return {
            'text': text,
            'label': item['label'],
            'speaker': item['speaker']
        }
```

---

## Bonnes Pratiques
- **Conserver les originales**: Toujours garder une copie des données originales.
- **Équilibrer l'augmentation**: Ne pas sur-augmenter certaines classes.
- **Valider la qualité**: Vérifier manuellement quelques exemples augmentés.
- **Documenter**: Garder une trace des techniques utilisées et des paramètres.